<a href="https://colab.research.google.com/github/Heyymant/Trexquant-Hangman/blob/main/hangman_hemant_submission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trexquant Interview Project (The Hangman Game)

* Copyright Trexquant Investment LP. All Rights Reserved.
* Redistribution of this question without written consent from Trexquant is prohibited

## Instruction:
For this coding test, your mission is to write an algorithm that plays the game of Hangman through our API server.

When a user plays Hangman, the server first selects a secret word at random from a list. The server then returns a row of underscores (space separated)—one for each letter in the secret word—and asks the user to guess a letter. If the user guesses a letter that is in the word, the word is redisplayed with all instances of that letter shown in the correct positions, along with any letters correctly guessed on previous turns. If the letter does not appear in the word, the user is charged with an incorrect guess. The user keeps guessing letters until either (1) the user has correctly guessed all the letters in the word
or (2) the user has made six incorrect guesses.

You are required to write a "guess" function that takes current word (with underscores) as input and returns a guess letter. You will use the API codes below to play 1,000 Hangman games. You have the opportunity to practice before you want to start recording your game results.

Your algorithm is permitted to use a training set of approximately 250,000 dictionary words. Your algorithm will be tested on an entirely disjoint set of 250,000 dictionary words. Please note that this means the words that you will ultimately be tested on do NOT appear in the dictionary that you are given. You are not permitted to use any dictionary other than the training dictionary we provided. This requirement will be strictly enforced by code review.

You are provided with a basic, working algorithm. This algorithm will match the provided masked string (e.g. a _ _ l e) to all possible words in the dictionary, tabulate the frequency of letters appearing in these possible words, and then guess the letter with the highest frequency of appearence that has not already been guessed. If there are no remaining words that match then it will default back to the character frequency distribution of the entire dictionary.

This benchmark strategy is successful approximately 18% of the time. Your task is to design an algorithm that significantly outperforms this benchmark.

###  Solution by Hemant Kumar

Downloading the canine model from the gdrive link. This is the sugggested way to download the neccessary model files

In [1]:
!gdown 1FL3Un_8ub82x0xWV8BCn7Bf8UGt4dhBT

Downloading...
From (original): https://drive.google.com/uc?id=1FL3Un_8ub82x0xWV8BCn7Bf8UGt4dhBT
From (redirected): https://drive.google.com/uc?id=1FL3Un_8ub82x0xWV8BCn7Bf8UGt4dhBT&confirm=t&uuid=5c2366a9-d195-4cc4-9b84-f00a51d65fad
To: /content/canine_best_model.tar.gz
100% 490M/490M [00:02<00:00, 205MB/s]


In [2]:
# Extract the model from the tar zipped file
!tar -xvzf canine_best_model.tar.gz


canine_best_model/
canine_best_model/config.json
canine_best_model/model.safetensors


Importing the relevant libraries

In [4]:

from transformers import CanineConfig,CanineTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Declaring the parameters of canine model to train and make predictions

In [5]:
# Tokens used in encoding the hangman states to text for input to the transformer
tokenizer = CanineTokenizer.from_pretrained('google/canine-s')
CANINE_MASK_TOKEN = tokenizer.mask_token
CANINE_SEP_TOKEN = tokenizer.sep_token
# Initialize the Canine-s model from a saved model or random initialisation
# configuration = CanineConfig()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# configuration.num_labels = 26
model = AutoModelForSequenceClassification.from_pretrained('./canine_best_model').to(device)
# Default training arguments for the model will be set here
# Learning rate and weight decay for pre-training or -play training
lr = 1e-5
weight_decay = 1e-4

# Relevant arguments when training_mode is set to -play
optimizer = None
criterion = None

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/854 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/657 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

Below functions are defined to fine tune the pretrained the model on a given data set.

```metric``` fucntion is used to calculate the relevant performance metrics/ \
```simulate_hangman_transformers``` mimics the hangman game to train the model on our choices of words.

In [6]:
import torch
import numpy as np
from transformers import CanineTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import f1_score, accuracy_score
from tqdm import tqdm
from matplotlib import pyplot as plt

# Helper Functions
def metrics(predictions, labels):
    """
    Compute metrics (accuracy and f1-score).
    """
    y_pred = np.argmax(predictions, axis=1)
    y_true = np.argmax(labels, axis=1)
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='macro')
    accuracy = accuracy_score(y_true, y_pred)
    return {'f1': f1_micro_average, 'accuracy': accuracy}

def simulate_hangman_transformers(model, tokenizer, word,device, max_wrong_guesses=6):
    """
    Simulate a game of Hangman.
    """
    word_idxs = {c: [i for i, x in enumerate(word) if x == c] for c in set(word)}
    all_letters = [chr(i) for i in range(ord('a'), ord('z') + 1)]
    guesses = {}
    encoded_word = "*" * len(word)
    num_guesses = 0
    outputs_model = []
    outputs_true = []

    while encoded_word != word and num_guesses < max_wrong_guesses:
        state = ''.join(guesses.keys()) + tokenizer.sep_token + encoded_word.replace('*', tokenizer.mask_token)
        encoded_state = tokenizer(state, padding="max_length", truncation=True, return_tensors="pt", max_length=64).to(device)
        output = model(**encoded_state)
        output_true = torch.zeros(26).to(device)
        for ch in word_idxs:
            if ch not in guesses:
                output_true[ord(ch) - ord('a')] = len(word_idxs[ch])
        out_sum = torch.sum(output_true)
        output_true = output_true / out_sum if out_sum.item() > 0 else output_true
        outputs_model.append(output.logits)
        outputs_true.append(output_true)

        output_np = output.logits.cpu().detach().numpy()
        guess_idx = np.argmax(output_np)
        guess = all_letters[guess_idx]
        while guess in guesses:
            output_np[0][guess_idx] = -float('inf')
            guess_idx = np.argmax(output_np)
            guess = all_letters[guess_idx]
        if guess in word_idxs:
            for i in word_idxs[guess]:
                encoded_word = encoded_word[:i] + guess + encoded_word[i + 1:]
        else:
            num_guesses += 1
        guesses[guess] = True

    return torch.vstack(outputs_model), torch.vstack(outputs_true), encoded_word == word



Stepsto load a save model (recommended) or train a new model

In [6]:
# Load Pretrained Model and Tokenizer
saved_model_path = None
if saved_model_path is not None:

        my_model = AutoModelForSequenceClassification.from_pretrained(saved_model_path).to(device)
else:
    configuration.num_labels = 26
    my_model = AutoModelForSequenceClassification.from_config(configuration).to(device)

tokenizer = CanineTokenizer.from_pretrained('google/canine-s')


# Training Parameters

lr = 1e-6
weight_decay = 1e-5
num_steps = 7
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
criterion = torch.nn.CrossEntropyLoss()


Creating a randomised training and validation dataset to train our model from a given file.

In [7]:
import random

# Step 1: Load the words from the file
file_path = "words_250000_train.txt"  # Path to the uploaded file
with open(file_path, "r") as file:
    words = file.read().splitlines()

# Step 2: Shuffle the dataset
random.seed(42)  # Set a seed for reproducibility
random.shuffle(words)

# Step 3: Split into training and validation datasets
train_ratio = 0.8
split_index = int(len(words) * train_ratio)
train_words = words[:split_index]
val_words = words[split_index:]

Uncomment the below code to fine tune the model on a specified given data(in our case ```words_250000,txt``` file.


**Note : Keep in mind the below training process is very slow and could take hours, even on a hardware accelerated environment. So save the model as well.**

In [8]:

# # Fine-tuning via Self-Play
# train_losses = []
# val_losses = []
# train_accuracies = []
# val_accuracies = []

# num_step_samples = len(train_words) // num_steps
# for step in range(num_steps):
#     total_loss, total_correct = 0, 0
#     my_model.train()
#     train_set = train_words[step * num_step_samples:(step + 1) * num_step_samples]

#     for word in tqdm(train_set, desc=f"Training Step {step + 1}/{num_steps}"):
#         outputs, outputs_true, correct = simulate_hangman_transformers(my_model, tokenizer, word, device=device)
#         loss = criterion(outputs, outputs_true)
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#         total_loss += loss.item()
#         total_correct += int(correct)

#     train_loss = total_loss / len(train_set)
#     train_acc = total_correct / len(train_set)
#     train_losses.append(train_loss)
#     train_accuracies.append(train_acc)

#     # Validation
#     total_val_loss, total_val_correct = 0, 0
#     model.eval()
#     for word in val_words:
#         outputs, outputs_true, correct = simulate_hangman_transformers(my_model, tokenizer, word, device=device)
#         loss = criterion(outputs, outputs_true)
#         total_val_loss += loss.item()
#         total_val_correct += int(correct)

#     val_loss = total_val_loss / len(val_words)
#     val_acc = total_val_correct / len(val_words)
#     val_losses.append(val_loss)
#     val_accuracies.append(val_acc)

#     print(f"Step {step + 1}: Train Loss={train_loss:.4f}, Train Acc={train_acc:.4f}, Val Loss={val_loss:.4f}, Val Acc={val_acc:.4f}")

#     # Save Checkpoint
#     my_model.save_pretrained(f"./canine-pretrained-hangman-checkpoint-{step}")

# # Plot Loss and Accuracy
# plt.plot(train_losses, label="Train Loss")
# plt.plot(val_losses, label="Validation Loss")
# plt.xlabel("Steps")
# plt.ylabel("Loss")
# plt.title("Learning Curve")
# plt.legend()
# plt.savefig('./self-play-loss.png')

# plt.plot(train_accuracies, label="Train Accuracy")
# plt.plot(val_accuracies, label="Validation Accuracy")
# plt.xlabel("Steps")
# plt.ylabel("Accuracy")
# plt.title("Accuracy vs Steps")
# plt.legend()
# plt.savefig('./self-play-accuracy.png')


Training Step 1/7:   1%|          | 154/25977 [00:42<1:57:56,  3.65it/s]


KeyboardInterrupt: 

The below function uses our trained model to make guesses for our hangman game

In [7]:
# Import necessary modules at the top of your script
from transformers import CanineTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np
import requests
import time
import collections
import re
import json
from urllib.parse import parse_qs

# Load the model and tokenizer once at the module level
tokenizer = CanineTokenizer.from_pretrained('google/canine-s')
CANINE_MASK_TOKEN = tokenizer.mask_token
CANINE_SEP_TOKEN = tokenizer.sep_token

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForSequenceClassification.from_pretrained('./canine_best_model')
model.to(device)
model.eval()  # Set model to evaluation mode

all_letters = [chr(i) for i in range(ord('a'), ord('z') + 1)]

def predict_next_letter(encoded_word, guessed_letters):
    # Build the set of guessed letters
    guessed_letters_set = set(guessed_letters)

    # Construct the model input state
    state = ''.join(guessed_letters) + CANINE_SEP_TOKEN + encoded_word

    # Tokenize the input state
    encoded_state = tokenizer(
        state,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
        max_length=128
    ).to(device)

    # Get model predictions
    with torch.no_grad():
        output = model(**encoded_state)
    logits = output.logits.cpu().numpy()[0]

    # Exclude already guessed letters
    for idx, letter in enumerate(all_letters):
        if letter in guessed_letters_set:
            logits[idx] = -float('inf')

    # Check if there are any valid letters left to guess
    if np.all(np.isneginf(logits)):
        return None  # All letters have been guessed

    # Select the letter with the highest logit score
    guess_idx = np.argmax(logits)
    guess_letter = all_letters[guess_idx]

    return guess_letter

In [8]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections
try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []

        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()

        self.current_dictionary = []
        self.n_word_dictionary = self.build_n_word_dictionary(self.full_dictionary)

    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com', 'https://sg.trexsim.com']

        data = {link: 0 for link in links}

        for link in links:

            requests.get(link)

            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link


    def build_dictionary(self, dictionary_file_location):
        with open(dictionary_file_location, "r") as text_file:
            full_dictionary = text_file.read().splitlines()
        return full_dictionary

    def build_n_word_dictionary(self, word_list):
        n_word_dict = {}
        for word in word_list:
            length = len(word)
            if length not in n_word_dict:
                n_word_dict[length] = []
            n_word_dict[length].append(word)
        return n_word_dict

    def vowel_count(self, clean_word):
        vowels = "aeiou"
        count = sum(1 for char in clean_word if char in vowels)
        return count / len(clean_word) if len(clean_word) > 0 else 0.0

    def func(self, new_dictionary):
        dictx = collections.Counter()
        for words in new_dictionary:
            temp = collections.Counter(words)
            for i in temp:
                temp[i] = 1
            dictx += temp
        return dictx

    def func2(self, n_word_dictionary, clean_word):
        new_dictionary = []
        length = len(clean_word)
        pattern = '^' + clean_word + '$'
        for dict_word in n_word_dictionary.get(length, []):
            if re.match(pattern, dict_word):
                new_dictionary.append(dict_word)
        return new_dictionary

    def guess(self, word):
        # Prepare the encoded word by removing spaces and replacing '_' with the mask token
        encoded_word = word.replace(' ', '').replace('_', CANINE_MASK_TOKEN)

        # Try to get the next letter prediction from the model
        guess_letter = predict_next_letter(encoded_word, self.guessed_letters)

        if guess_letter is None:
            # Fallback to the original functionality
            guess_letter = self.fallback_guess(word)

        if guess_letter is None:
            # Handle the case when no valid letters are left to guess
            raise ValueError("No valid letters left to guess.")

        return guess_letter

    def fallback_guess(self, word):
        # This method contains the original guess functionality provided
        clean_word = word[::2].replace("_", ".")
        len_word = len(clean_word)
        current_dictionary = self.current_dictionary
        new_dictionary = []

        for dict_word in current_dictionary:
            if len(dict_word) != len_word:
                continue

            if re.match(clean_word, dict_word):
                new_dictionary.append(dict_word)

        self.current_dictionary = new_dictionary
        c = self.func(new_dictionary)
        sorted_letter_count = c.most_common()

        guess_letter = None

        for letter, instance_count in sorted_letter_count:
            if letter not in self.guessed_letters:
                if letter in "aeiou" and self.vowel_count(clean_word) > 0.55:
                    self.guessed_letters.append(letter)
                    continue
                guess_letter = letter
                break

        if guess_letter is None:
            new_dictionary = self.func2(self.n_word_dictionary, clean_word)
            c = self.func(new_dictionary)
            sorted_letter_count = c.most_common()
            for letter, instance_count in sorted_letter_count:
                if letter not in self.guessed_letters:
                    if letter in "aeiou" and self.vowel_count(clean_word) > 0.55:
                        self.guessed_letters.append(letter)
                        continue
                    guess_letter = letter
                    break

        if guess_letter is None:
            x = int(len(clean_word) / 2)
            if x >= 3:
                c = collections.Counter()
                for i in range(len(clean_word) - x + 1):
                    s = clean_word[i:i + x]
                    new_dictionary = self.func2(self.n_word_dictionary, s)
                    temp = self.func(new_dictionary)
                    c += temp
                sorted_letter_count = c.most_common()
                for letter, instance_count in sorted_letter_count:
                    if letter not in self.guessed_letters:
                        guess_letter = letter
                        break

        if guess_letter is None:
            x = int(len(clean_word) / 3)
            if x >= 3:
                c = collections.Counter()
                for i in range(len(clean_word) - x + 1):
                    s = clean_word[i:i + x]
                    new_dictionary = self.func2(self.n_word_dictionary, s)
                    temp = self.func(new_dictionary)
                    c += temp
                sorted_letter_count = c.most_common()
                for letter, instance_count in sorted_letter_count:
                    if letter not in self.guessed_letters:
                        guess_letter = letter
                        break

        if guess_letter is None:
            sorted_letter_count = self.full_dictionary_common_letter_sorted
            for letter, instance_count in sorted_letter_count:
                if letter not in self.guessed_letters:
                    if letter in "aeiou" and self.vowel_count(clean_word) > 0.55:
                        self.guessed_letters.append(letter)
                        continue
                    guess_letter = letter
                    break

        return guess_letter


    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary

        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)

                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))

                try:
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e

                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"

    def my_status(self):
        return self.request("/my_status", {})

    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        try:
            # response = self.session.request(
            response = requests.request(
                method or "GET",
                self.hangman_url + path,
                timeout=self.timeout,
                params=args,
                data=post_args)
        except requests.HTTPError as e:
            response = json.loads(e.read())
            raise HangmanAPIError(response)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result

class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

# API Usage Examples

## To start a new game:
1. Make sure you have implemented your own "guess" method.
2. Use the access_token that we sent you to create your HangmanAPI object.
3. Start a game by calling "start_game" method.
4. If you wish to test your function without being recorded, set "practice" parameter to 1.
5. Note: You have a rate limit of 20 new games per minute. DO NOT start more than 20 new games within one minute.

In [10]:
api = HangmanAPI(access_token="ee5b98942cee4529cb87728c3719c8", timeout=2000)

## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [14]:
for i in range(100):
    print(f"playing {i+1}th game:")
    api.start_game(practice=1,verbose=True)

    time.sleep(0.5)

playing 1th game:
Successfully start a new game! Game ID: a1a5e21ada9a. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ _ _ _ _ .
Guessing letter: i
Sever response: {'game_id': 'a1a5e21ada9a', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ i _ _ _ _ _ i _ _ '}
Guessing letter: n
Sever response: {'game_id': 'a1a5e21ada9a', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ i _ _ _ _ _ i _ n '}
Guessing letter: t
Sever response: {'game_id': 'a1a5e21ada9a', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ i t _ _ _ t i _ n '}
Guessing letter: a
Sever response: {'game_id': 'a1a5e21ada9a', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ a _ _ i t _ _ a t i _ n '}
Guessing letter: o
Sever response: {'game_id': 'a1a5e21ada9a', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ a _ _ i t _ _ a t i o n '}
Guessing letter: r
Sever response: {'game_id': 'a1a5e21ada9a', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ a _ _ i t r _ a t i o n '}
Guessing letter: 

In [15]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
practice_success_rate = total_practice_successes / total_practice_runs
print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))

run 1859 practice games out of an allotted 100,000. practice success rate so far = 0.446


## Playing recorded games:
Please finalize your code prior to running the cell below. Once this code executes once successfully your submission will be finalized. Our system will not allow you to rerun any additional games.

Please note that it is expected that after you successfully run this block of code that subsequent runs will result in the error message "Your account has been deactivated".

Once you've run this section of the code your submission is complete. Please send us your source code via email.

In [17]:
for i in range(1000):
    print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    api.start_game(practice=0,verbose=False)

    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

Playing  0  th game
Playing  1  th game
Playing  2  th game
Playing  3  th game
Playing  4  th game
Playing  5  th game
Playing  6  th game
Playing  7  th game
Playing  8  th game
Playing  9  th game
Playing  10  th game
Playing  11  th game
Playing  12  th game
Playing  13  th game
Playing  14  th game
Playing  15  th game
Playing  16  th game
Playing  17  th game
Playing  18  th game
Playing  19  th game
Playing  20  th game
Playing  21  th game
Playing  22  th game
Playing  23  th game
Playing  24  th game
Playing  25  th game
Playing  26  th game
Playing  27  th game
Playing  28  th game
Playing  29  th game
Playing  30  th game
Playing  31  th game
Playing  32  th game
Playing  33  th game
Playing  34  th game
Playing  35  th game
Playing  36  th game
Playing  37  th game
Playing  38  th game
Playing  39  th game
Playing  40  th game
Playing  41  th game
Playing  42  th game
Playing  43  th game
Playing  44  th game
Playing  45  th game
Playing  46  th game
Playing  47  th game
Pl

HangmanAPIError: {'error': 'You have reached 1000 of games', 'status': 'denied'}

## To check your game statistics
1. Simply use "my_status" method.
2. Returns your total number of games, and number of wins.

In [18]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
success_rate = total_recorded_successes/total_recorded_runs
print('overall success rate = %.3f' % success_rate)

overall success rate = 0.589
